In [1]:
import os
import requests

run code cell below to retrieve updated ONI data

In [2]:
url = "https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt"
path = "data/oni.ascii.txt"

os.makedirs(os.path.dirname(path), exist_ok=True)
response = requests.get(url)
with open(path, 'w') as fd:
    fd.write(response.text)

print(f"File saved to {path}")

File saved to data/oni.ascii.txt


In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

oni = pd.read_csv(path, sep=r'\s+')
oni.head()

,SEAS,YR,TOTAL,ANOM
0,DJF,1950,24.72,-1.53
1,JFM,1950,25.17,-1.34
2,FMA,1950,25.75,-1.16
3,MAM,1950,26.12,-1.18
4,AMJ,1950,26.32,-1.07


In [4]:
oni_copy = oni.copy()
oni_copy['SEAS_YR'] = oni_copy['SEAS'] + ' ' + oni_copy['YR'].astype(str)
oni_copy.drop(columns=['SEAS', 'YR'], inplace=True)

In [5]:
import pandas as pd
import plotly.graph_objects as go

In [6]:
def plot_oni(oni):

    color = []

    for _, row in oni.iterrows():
        if row['ANOM'] > 0:
            color.append('red')
        else:
            color.append('blue')

    fig = go.Bar(x=oni['SEAS_YR'], y=oni['ANOM'], marker_color=color)
    figure = go.Figure(data=fig)
    figure.update_layout(title_text='Oceanic Niño Index (ONI)')
    figure.show()    

In [7]:
plot_oni(oni_copy)